# Bicycle data melbourne analysis

In [2]:
import pandas as pd
import numpy as np
import os
from collections import Counter
from tqdm.notebook import tqdm

## Data validation 

The bicycle data is delivered in a large number of CSV files, without an accompanying data dictionary.  To be sure the same columns are shared by all CSVs, such that they can be appended to a master file, we iterate over them and tally up the unique column combinations.  Ideally, the results will be a single combination of columns with a tally count the length of the number of CSV files.  Let's see!

In [2]:
#rootdir ='../data'
#tally = pd.DataFrame(columns=["count","len"])
#counter = 0
#for subdir, dirs, files in tqdm(os.walk(rootdir)):
#    for file in files:
#        if ('.csv' in file) and ('.zip' not in file):
#            # read in CSV, if it contains records (which at least one doesn't!)
#            file_path = os.path.join(subdir,file)
#            if os.path.getsize(file_path) > 0:
#                df = pd.read_csv(os.path.join(subdir,file))
#                # store list of columns in variable df_columns as a string
#                df_columns = f"{df.columns.to_list()}"
#                # if CSV columns string is in the tally index, increment this
#                if df_columns in tally.index:
#                    tally[tally.index==df_columns] += 1
#                # otherwise add CSV columns string to the tally index
#                else:
#                    tally.loc[df_columns] = 1
#                # increment a counter; athough theoretically this should only sum to the sum of tallys!
#                counter+=1
#
#print(counter)

In [8]:
tally

,count
"['DATA_TYPE', 'TIS_DATA_REQUEST', 'SITE_XN_ROUTE', 'LOC_LEG', 'DATE', 'TIME', 'CLASS', 'LANE', 'SPEED', 'WHEELBASE', 'HEADWAY', 'GAP', 'AXLE', 'AXLE_GROUPING', 'RHO', 'VEHICLE', 'DIRECTION']",13229


## Start Dask dashboard
The bicycle data is comprised of more than 16,000 CSV files; this is a very large multi-gigabyte dataset, and so rather than use Pandas which will struggle to load and manipulate the data, [Dask](https://dask.org) could provide a good option for parallel processing.

The use of the dashboard is optional, though.  So, commented out for now.

In [1]:
# from dask.distributed import Client
# client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
# client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://192.168.1.4:8787/status,
Dashboard: http://192.168.1.4:8787/status,Workers: 1
Total threads: 4,Total memory: 1.86 GiB
Status: running,Using processes: False
Comm: inproc://192.168.1.4/12364/1,Workers: 1
Dashboard: http://192.168.1.4:8787/status,Total threads: 4
Started: Just now,Total memory: 1.86 GiB
Comm: inproc://192.168.1.4/12364/4,Total threads: 4
Dashboard: http://192.168.1.4:55991/status,Memory: 1.86 GiB
Nanny: None,


## Creating a master dataframe

Now that we know that all of the CSVs share the same column names, we will join the various files together to create a master dataframe to run the analysis on. 

In [3]:
data_years = ['Bicycle_Volume_Speed_2017',
'Bicycle_Volume_Speed_2018',
'Bicycle_Volume_Speed_2019',
'Bicycle_Volume_Speed_2020',
'Bicycle_Volume_Speed_2021']

rootdir = f'../data/{data_years[0]}'
csv_files = []

for subdir, dirs, files in tqdm(os.walk(rootdir),desc="Getting CSV file paths...",unit="CSVs"):
    for file in files:
        if ('.csv' in file) and ('.zip' not in file):
            # record filepaths of CSVs containing records
            file_path = os.path.join(subdir,file)
            if os.path.getsize(file_path) > 0:
                csv_files.append(os.path.join(subdir,file))

print(f"Identified the locations of {len(csv_files)} valid CSV files to compile!")

Getting CSV file paths...: 0CSVs [00:00, ?CSVs/s]

Identified the locations of 1430 valid CSV files to compile!


In [18]:
dfs=[]
for csv in tqdm(csv_files,desc=f"Reading csv files for {rootdir}...",unit="CSVs"):
    csv_df = pd.read_csv(csv, index_col=None, header=0)
    dfs.append(csv_df)
    del csv_df

dfs = pd.concat(dfs, axis=0, ignore_index=True)

Reading csv files for ../data/Bicycle_Volume_Speed_2017...:   0%|          | 0/1430 [00:00<?, ?CSVs/s]

In [21]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9989156 entries, 0 to 9989155
Data columns (total 17 columns):
 #   Column            Dtype  
---  ------            -----  
 0   DATA_TYPE         object 
 1   TIS_DATA_REQUEST  object 
 2   SITE_XN_ROUTE     object 
 3   LOC_LEG           object 
 4   DATE              object 
 5   TIME              object 
 6   CLASS             object 
 7   LANE              object 
 8   SPEED             float64
 9   WHEELBASE         float64
 10  HEADWAY           float64
 11  GAP               float64
 12  AXLE              object 
 13  AXLE_GROUPING     object 
 14  RHO               float64
 15  VEHICLE           object 
 16  DIRECTION         object 
dtypes: float64(5), object(12)
memory usage: 1.3+ GB


In [4]:
import dask.dataframe as dd
# manually specifying datatypes to increase likelihood of successful read...
df = dd.read_csv(csv_files, 
                 dtype={'DATA_TYPE': 'str',
                        'TIS_DATA_REQUEST': 'int64',
                        'SITE_XN_ROUTE': 'int64',
                        'LOC_LEG': 'int64',
                        'DATE': 'object',
                        'TIME': 'object',
                        'CLASS': 'int64',
                        'LANE': 'int64',
                        'SPEED': 'float',
                        'WHEELBASE': 'float',
                        'HEADWAY': 'float',
                        'GAP': 'float',
                        'AXLE': 'float64',
                        'AXLE_GROUPING': 'float64',
                        'RHO': 'float64',
                        'VEHICLE': 'str',
                        'DIRECTION': 'str'}
                )


In [5]:
df.npartitions

1430

In [14]:
print(f"Total dataframe size: {df.partitions[0].memory_usage(deep=True).compute().sum()/(10**6)}Mb")

64.657216

In [7]:
# repartition dataframe into fewer, larger chunks
df = df.repartition(npartitions=30)

In [8]:
df.index.size.compute(processor='threads')

9989156

In [16]:
nrows.compute()

KeyboardInterrupt: 

## Summary statistics

Let's look at the summary statistics for the master dataframe! First we'll look at what type of data is included in each column. Then we can see can check the max/min values, and the distribution of the data, to see if there are any outlying data points. 

In [32]:
#Checking data types in each column

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4925 entries, 0 to 4924
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   DATA_TYPE         4925 non-null   object 
 1   TIS_DATA_REQUEST  4925 non-null   int64  
 2   SITE_XN_ROUTE     4925 non-null   int64  
 3   LOC_LEG           4925 non-null   int64  
 4   DATE              4925 non-null   object 
 5   TIME              4925 non-null   object 
 6   CLASS             4925 non-null   int64  
 7   LANE              4925 non-null   int64  
 8   SPEED             4925 non-null   float64
 9   WHEELBASE         4925 non-null   float64
 10  HEADWAY           4925 non-null   float64
 11  GAP               4925 non-null   float64
 12  AXLE              4925 non-null   int64  
 13  AXLE_GROUPING     4925 non-null   int64  
 14  RHO               4925 non-null   float64
 15  VEHICLE           4925 non-null   object 
 16  DIRECTION         4925 non-null   object 


In [34]:
#Summary statistics

df.describe()

,TIS_DATA_REQUEST,SITE_XN_ROUTE,LOC_LEG,CLASS,LANE,SPEED,WHEELBASE,HEADWAY,GAP,AXLE,AXLE_GROUPING,RHO
count,4925.0,4925.0,4925.000000,4925.0,4925.000000,4925.000000,4925.000000,4925.000000,4925.000000,4925.0,4925.0,4925.000000
mean,208.0,10223.0,59443.482437,15.0,0.482437,18.764264,1.001401,473.021401,472.778518,2.0,1.0,0.984083
std,0.0,0.0,0.499742,0.0,0.499742,7.449773,0.142950,2259.648522,2259.641277,0.0,0.0,0.081180
min,208.0,10223.0,59443.000000,15.0,0.000000,1.400000,0.000000,0.000000,0.000000,2.0,1.0,0.200000
25%,208.0,10223.0,59443.000000,15.0,0.000000,13.200000,1.000000,5.600000,5.400000,2.0,1.0,1.000000
50%,208.0,10223.0,59443.000000,15.0,0.000000,18.500000,1.000000,109.100000,108.700000,2.0,1.0,1.000000
75%,208.0,10223.0,59444.000000,15.0,1.000000,23.900000,1.100000,362.800000,361.700000,2.0,1.0,1.000000
max,208.0,10223.0,59444.000000,15.0,1.000000,120.500000,4.000000,35194.600000,35194.400000,2.0,1.0,1.200000


In [44]:
os.path.getsize('../data\Bicycle_Volume_Speed_2020\Black Rock - Bicycle volume data 2020\IND_D5555_X32021.csv.20201228')

0